# 5-Layers Specialisation

## 1st way

In [1]:
## Original packages
import backbone
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from torch.func import functional_call, vmap, vjp, jvp, jacrev
from methods.meta_template import MetaTemplate
import math
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR
import warnings
from torch.distributions import MultivariateNormal
import warnings

In [2]:
class simple_netC_0hl(nn.Module):
    def __init__(self):
        super(simple_netC_0hl, self).__init__()
        self.layer1 = nn.Linear(1600, 5)
        
    def forward(self, x):
        out = self.layer1(x)
        return out

net = simple_netC_0hl()

In [3]:
c=0

def compute_jacobian(inputs):   # i is the class label, and corresponds to the output targeted
    """
    Return the jacobian of a batch of inputs, thanks to the vmap functionality
    """
    net.zero_grad()
    params = {k: v for k, v in net.named_parameters()}

    def fnet_single(params, x):
        # Make sure output has the right dimensions
        return functional_call(net, params, (x.unsqueeze(0),)).squeeze(0)[c]

    jac = vmap(jacrev(fnet_single), (None, 0))(params, inputs)
    jac_values = jac.values()

    reshaped_tensors = []
    for j in jac_values:
        if len(j.shape) == 3:  # For layers with weights
            # Flatten parameters dimensions and then reshape
            flattened = j.flatten(start_dim=1)  # Flattens to [batch, params]
            reshaped = flattened.T  # Transpose to align dimensions as [params, batch]
            reshaped_tensors.append(reshaped)
        elif len(j.shape) == 2:  # For biases or single parameter components
            reshaped_tensors.append(j.T)  # Simply transpose

    # Concatenate all the reshaped tensors into one large matrix
    return torch.cat(reshaped_tensors, dim=0).T

In [4]:
x = torch.empty(3, 1600)

# Fill each row of the tensor with the row index
for i in range(3):
    x[i] = i

print(x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [2., 2., 2.,  ..., 2., 2., 2.]])


In [5]:
print(compute_jacobian(x))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [2., 2., 2.,  ..., 0., 0., 0.]])


In [6]:
# Constructing the specialisation matrix
with torch.no_grad():
    spe = net(x)
print(spe.shape)
print(spe)

torch.Size([3, 5])
tensor([[ 0.0078,  0.0053,  0.0200,  0.0206,  0.0170],
        [ 0.5688,  0.2445,  0.5690, -0.3080,  0.3996],
        [ 1.1297,  0.4838,  1.1180, -0.6366,  0.7822]])


In [7]:
# Generate a single random number between 0 and n_classes
random_class = torch.randint(low=0, high=5, size=(1,))

print(random_class)

tensor([0])


In [8]:
col = spe[:, random_class].flatten()
print(col)

tensor([0.0078, 0.5688, 1.1297])


In [9]:
with torch.no_grad():
    softmax_col = F.softmax(col, dim=0)

print(softmax_col)

tensor([0.1717, 0.3009, 0.5273])


In [10]:
# Use multinomial to pick an index based on the weights
# The second argument 'num_samples' is the number of indices to sample
# 'replacement=True' allows picking the same index more than once if num_samples > 1
random_index = torch.multinomial(softmax_col, num_samples=1, replacement=True)

print(random_index)


tensor([2])


In [11]:
print(f"So input number {random_index[0]} will have class number {random_class[0]}")

So input number 2 will have class number 0


In [12]:
#Now updating the matrix :
i = random_index[0]
j = random_class[0]
# Remove the ith row
new_spe = torch.cat((spe[:i], spe[i+1:]))

# Remove the jth column
new_spe = torch.cat((new_spe[:, :j], new_spe[:, j+1:]), dim=1)

print("Modified Matrix:\n", new_spe)

Modified Matrix:
 tensor([[ 0.0053,  0.0200,  0.0206,  0.0170],
        [ 0.2445,  0.5690, -0.3080,  0.3996]])


In [13]:
#repeat the process

x = torch.empty(5, 1600)
for i in range(5):
    x[i] = i
    
# Constructing the specialisation matrix
with torch.no_grad():
    spe = net(x)

classes = torch.tensor([0, 1, 2, 3, 4])
for _ in range(5):
    # Pick a class randomly with equal probability
    random_class = classes[torch.randint(low=0, high=len(classes), size=(1,))]
    col = spe[:, random_class].flatten()
    with torch.no_grad():
        softmax_col = F.softmax(col, dim=0)
    random_index = torch.multinomial(softmax_col, num_samples=1, replacement=True)
    
    print(f"Input number {random_index[0]} will have class number {random_class[0]}")
    
    i = random_index[0]
    j = random_class[0]
    
    # Remove the ith row
    spe[i] = float('-inf')
    
    # can't pick the jth class anymore
    mask = classes != random_class
    classes = classes[mask]
    

Input number 2 will have class number 1
Input number 4 will have class number 0
Input number 3 will have class number 4
Input number 1 will have class number 3
Input number 0 will have class number 2


## 3rd option

In [14]:
#repeat the process

x = torch.empty(5, 1600)
for i in range(5):
    x[i] = i
    
# Constructing the specialisation matrix
with torch.no_grad():
    spe = net(x)

flattened_spe = spe.flatten()
for _ in range(5):
    #Take the softmax of all the elements in the matrix
    with torch.no_grad():
        softmax_matrix = F.softmax(flattened_spe, dim=0)

    rd_element_idx = torch.multinomial(softmax_matrix, num_samples=1, replacement=True)
    rd_elemt = rd_element_idx // 5 # Indice of the row
    rd_class = rd_element_idx % 5 # Indice of the column
    indices_1 = torch.tensor([5 * i + rd_class for i in range(5)])
    indices_2 = torch.tensor([i + rd_elemt * 5 for i in range(5)])

    # Combine indices from both calculations, ensuring uniqueness if necessary
    all_indices = torch.cat((indices_1, indices_2)).unique()
    flattened_spe[all_indices] = float('-inf')
    print(f"Input number {rd_elemt[0]} will have class number {rd_class[0]}")
    

Input number 3 will have class number 0
Input number 1 will have class number 1
Input number 4 will have class number 4
Input number 2 will have class number 2
Input number 0 will have class number 3
